In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import pandas as pd


class IMDbScraper:
    """Class to scrape IMDb movie details for different genres."""

    def __init__(self):
        """Initialize the Chrome WebDriver."""
        self.driver = webdriver.Chrome()
        self.base_url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31"
        self.driver.get(self.base_url)
        self.driver.maximize_window()
        time.sleep(3)  

    def click_element(self, xpath):
        """Safely clicks an element using its XPath."""
        try:
            element = WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.XPATH, xpath)))
            self.driver.execute_script("arguments[0].scrollIntoView(true);", element)
            time.sleep(1)  
            self.driver.execute_script("arguments[0].click();", element)
            time.sleep(3) 
        except Exception as e:
            print(f"Error clicking element: {xpath}, {e}")

    def click_load_more(self):
        """Attempts to click the 'Load More' button to load additional movies."""
        while True:
            try:
                load_more_xpath = '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span'
                element = WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.XPATH, load_more_xpath)))
                self.driver.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(1)
                self.driver.execute_script("arguments[0].click();", element)
                time.sleep(3)
                print("Clicked 'Load More' button.")
            except Exception:
                print("No 'Load More' button found or all movies loaded.")
                break

    def get_genre_from_url(self):
        """Extracts the genre from the current IMDb URL."""
        time.sleep(2) 
        current_url = self.driver.current_url
        match = re.search(r'genres=([\w-]+)', current_url)
        return match.group(1).capitalize() if match else "Unknown"

    @staticmethod
    def convert_duration_to_minutes(duration_text):
        hours = 0
        minutes = 0
        if match := re.search(r'(\d+)h', duration_text):
            hours = int(match.group(1))
        if match := re.search(r'(\d+)m', duration_text):
            minutes = int(match.group(1))
        return (hours * 60) + minutes

    def extract_movies(self):
        """Extracts movie details and saves them to a CSV file."""
        genre_name = self.get_genre_from_url()
        titles, ratings, votings, durations, genres = [], [], [], [], []

        movie_items = self.driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')

        for movie_item in movie_items:
            try:
                title = movie_item.find_element(By.XPATH, './div/div/div/div[1]/div[2]/div[1]/a/h3').text.split('. ', 1)[-1]
                rating = movie_item.find_element(By.XPATH, './div/div/div/div[1]/div[2]/span/div/span/span[1]').text
                voting = movie_item.find_element(By.XPATH, './div/div/div/div[1]/div[2]/span/div/span/span[2]').text.replace("(", "").replace(")", "")
                duration_text = movie_item.find_element(By.XPATH, './div/div/div/div[1]/div[2]/div[2]/span[2]').text

                duration = self.convert_duration_to_minutes(duration_text)

                if "K" in voting:
                    voting = int(float(voting.replace("K", "")) * 1000)
                elif "M" in voting:
                    voting = int(float(voting.replace("M", "")) * 1000000)
                elif voting.strip().isdigit():
                    voting = int(voting)
                else:
                    voting = "N/A"

                if rating != "N/A" and voting != "N/A" and duration > 0:
                    titles.append(title)
                    ratings.append(rating)
                    votings.append(voting)
                    durations.append(duration)
                    genres.append(genre_name)

            except Exception:
                pass  

        df = pd.DataFrame({
            "Title": titles,
            "Genre": genres,
            "Rating": ratings,
            "Votes": votings,
            "Duration (Minutes)": durations
        })

        filename = f"{genre_name}_movies.csv"
        df.to_csv(filename, index=False, encoding="utf-8")
        print(f"Saved data to {filename}")

    def run_scraper(self):
        genre_button_xpath = '//*[@id="genreAccordion"]/div[1]/label/span[1]/div'
        genre_xpaths = {
            "Action": '//*[@id="accordion-item-genreAccordion"]/div/section/button[1]/span',
            "Adventure": '//*[@id="accordion-item-genreAccordion"]/div/section/button[2]/span',
            "Animation": '//*[@id="accordion-item-genreAccordion"]/div/section/button[3]/span',
            "Crime": '//*[@id="accordion-item-genreAccordion"]/div/section/button[6]/span',
            "Family": '//*[@id="accordion-item-genreAccordion"]/div/section/button[9]/span',
            "History": '//*[@id="accordion-item-genreAccordion"]/div/section/button[13]/span'
        }

        self.click_element(genre_button_xpath)

        for genre, xpath in genre_xpaths.items():
            self.click_element(xpath)
            self.click_load_more()
            self.extract_movies()
            self.click_element(xpath)

        self.click_element(genre_button_xpath)
        self.driver.quit()


def main():
    """Main function to execute the IMDb scraper."""
    scraper = IMDbScraper()
    scraper.run_scraper()


if __name__ == "__main__":
    main()


Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
No 'Load More' button found or all movies loaded.
Saved data to Action_movies.csv
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' bu

In [6]:
import os
import pandas as pd

def get_csv_files(directory=".", suffix="_movies.csv"):
    """Retrieves all CSV files with a specified suffix in the given directory."""
    return [file for file in os.listdir(directory) if file.endswith(suffix)]

def merge_csv_files(csv_files, output_filename="IMDb_2024_All_Movies.csv"):
    """Merges multiple CSV files into a single CSV file."""
    if not csv_files:
        print("No CSV files found to merge.")
        return
    
    dataframes = []
    
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            dataframes.append(df)
            print(f"Successfully loaded: {file}")
        except Exception as e:
            print(f"Error reading {file}: {e}")
    
    if dataframes:
        merged_df = pd.concat(dataframes, ignore_index=True)
        merged_df.to_csv(output_filename, index=False, encoding="utf-8")
        print(f"Merged {len(csv_files)} files into '{output_filename}'.")
    else:
        print("No valid data to merge.")

def main():
    """Main function to execute the merging process."""
    csv_files = get_csv_files()
    merge_csv_files(csv_files)

if __name__ == "__main__":
    main()


Successfully loaded: Action_movies.csv
Successfully loaded: Adventure_movies.csv
Successfully loaded: Animation_movies.csv
Successfully loaded: Crime_movies.csv
Successfully loaded: Family_movies.csv
Successfully loaded: History_movies.csv
Merged 6 files into 'IMDb_2024_All_Movies.csv'.


In [22]:
import pandas as pd
import mysql.connector

def connect_to_tidb():
    """Establish connection to TiDB."""
    return mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2Djg5GyoVwWC4gN.root",
        password="JWYIDlQjiVFU2pUD"
    )

def create_database_and_table(mycursor):
    """Create database and table if they do not exist."""
    mycursor.execute("CREATE DATABASE IF NOT EXISTS IMDB")
    mycursor.execute("USE IMDB")
    mycursor.execute("""
        CREATE TABLE IF NOT EXISTS IMDB.IMDb_Movies (
            Title VARCHAR(255),
            Genre VARCHAR(50),
            Rating FLOAT,
            Votes INT,
            Duration_Minutes INT
        )
    """)

def insert_data(mycursor, connection, df):
    """Insert movie data into the IMDb_Movies table."""
    insert_query = """
        INSERT INTO IMDb_Movies (Title, Genre, Rating, Votes, Duration_Minutes)
        VALUES (%s, %s, %s, %s, %s)
    """
    data = df[['Title', 'Genre', 'Rating', 'Votes', 'Duration (Minutes)']].values.tolist()
    mycursor.executemany(insert_query, data)
    connection.commit()
    print(f"Inserted {len(df)} records into IMDb_Movies.")

def main():
    """Main function to execute database operations."""
    df = pd.read_csv("IMDb_2024_All_Movies.csv")
    connection = connect_to_tidb()
    mycursor = connection.cursor(buffered=True)
    
    create_database_and_table(mycursor)
    insert_data(mycursor, connection, df)
    
    mycursor.close()
    connection.close()
    print("Database connection closed.")

if __name__ == "__main__":
    main()

Inserted 2475 records into IMDb_Movies.
Database connection closed.
